In [ ]:
# -*- coding:utf-8 -*-
import tushare as ts
import matplotlib.pyplot as plt
#import matplotlib.finance as mpf
import mpl_finance as mpf
import matplotlib.ticker as ticker

import datetime
import numpy as np
import pandas as pd
import time

from pyecharts.charts import Kline
from pyecharts.charts import Line
from pyecharts.charts import Bar
from pyecharts.charts import Grid

from pandas import DataFrame, Series

from pyecharts import options as opts


In [ ]:
def n_days_ago(n):
    today=datetime.date.today()
    ndays_ago=today-datetime.timedelta(n)
    return str(ndays_ago)

def hhv(s, n):
    return Series.rolling(s, n).max()

def llv(s, n):
    return Series.rolling(s, n).min()

    
#ichimoku
def ichimoku(s, n1=9, n2=26, n3=52):

    #average of N-day high and N-day low
    conv = (hhv(s, n1) + llv(s, n1)) / 2

    #mid point of the latest 26 days
    base = (hhv(s, n2) + llv(s, n2)) / 2
    
    #mid-point between the first 2 lines,  and plot 26 periods ahead
    spana = (conv + base) / 2
    
    #mid-point between the 52-period low and  52-period high, and plot 26 periods
    spanb = (hhv(s, n3) + llv(s, n3)) / 2
    
    k = s

    #Lspan is closing price, and plot 26 periods in the past
    return DataFrame(dict(k=k,conv=conv, base=base, spana=spana.shift(n2),
                          spanb=spanb.shift(n2), lspan=s.shift(-n2)))

In [ ]:
start_date = '2018-07-01'
now = int(time.time())
timeArray = time.localtime(now)
Time = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
end_date =  Time 
ndays = 360
stock_selected = '600487'
MAX_SMA=180
start_date = n_days_ago(ndays + MAX_SMA)
ktype = '60'

In [ ]:
df = ts.get_k_data(stock_selected,start_date,end_date)
#v_kline = np.array(df[['open','close','low','high']][MAX_SMA:])
date_kline = []
for i,j in enumerate(np.array(df[['date']][MAX_SMA:])):
    #print(j)
    temp_str = str(j[0]).replace('-0','/')
    date_kline.append(temp_str.replace('-','/'))

v_kline= []
for i,j in enumerate(np.array(df[['open','close','low','high']][MAX_SMA:])):
    #print(j)
    v_kline.append(list(j))

SMA5= []
for i,j in enumerate(np.array(pd.Series.rolling(df[['close']],5).mean()[MAX_SMA:])):
    #print(j)
    SMA5.append(list(j))
SMA55= []
for i,j in enumerate(np.array(pd.Series.rolling(df[['close']],55).mean()[MAX_SMA:])):
    #print(j)
    SMA55.append(list(j))
SMA100= []
for i,j in enumerate(np.array(pd.Series.rolling(df[['close']],100).mean()[MAX_SMA:])):
    #print(j)
    SMA100.append(list(j))
    
df_ich =  ichimoku(df['close'])

base= list(np.array(df_ich['base'])[MAX_SMA:])
conv= list(np.array(df_ich['conv'])[MAX_SMA:])
spana= list(np.array(df_ich['spana'])[MAX_SMA:])
spanb= list(np.array(df_ich['spanb'])[MAX_SMA:])

v_volume = np.array(df['volume'][MAX_SMA:])

In [ ]:
kline=Kline().add_xaxis(date_kline).add_yaxis('Kline',list(v_kline)).set_global_opts(
            xaxis_opts=opts.AxisOpts(is_scale=True),
            yaxis_opts=opts.AxisOpts(
                is_scale=True,
                splitarea_opts=opts.SplitAreaOpts(
                    is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                ),
            ),
            datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%")],
            title_opts=opts.TitleOpts(title="Kline"),
        )

#kline.render_notebook()

volume_bar = Bar().add_xaxis(date_kline).add_yaxis("", list(v_volume),label_opts=opts.LabelOpts(is_show=False)).set_global_opts(
            title_opts=opts.TitleOpts(title="", pos_top="48%"),
            legend_opts=opts.LegendOpts(pos_top="48%"),
        )
#volume_bar.render_notebook()

In [ ]:
sma_line = Line().add_xaxis(date_kline).add_yaxis("SMA5",list(SMA5),is_smooth=True,label_opts=opts.LabelOpts(is_show=False),
          linestyle_opts=opts.LineStyleOpts(color="red", width=1))
sma_line.add_yaxis("base",list(base),is_smooth=True,label_opts=opts.LabelOpts(is_show=False))
sma_line.add_yaxis("conv",list(conv),is_smooth=True,label_opts=opts.LabelOpts(is_show=False),
                  linestyle_opts=opts.LineStyleOpts(color="black", width=3))
sma_line.add_yaxis("spana",list(spana),is_smooth=True,
                   label_opts=opts.LabelOpts(is_show=False),
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.3),
                  linestyle_opts=opts.LineStyleOpts(color="red", width=3))
sma_line.add_yaxis("spanb",list(spanb),is_smooth=True,
                   label_opts=opts.LabelOpts(is_show=False),
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.3),
                   linestyle_opts=opts.LineStyleOpts(color="green", width=3))
kline.overlap(sma_line).render_notebook()
grid = (
        Grid()
        .add(kline, grid_opts=opts.GridOpts(pos_bottom="40%"))
        .add(volume_bar, grid_opts=opts.GridOpts(pos_top="70%"))
)
grid.render_notebook()

In [ ]:
r_hex = '#dc2624'     # red,       RGB = 220,38,36
dt_hex = '#2b4750'    # dark teal, RGB = 43,71,80
tl_hex = '#45a0a2'    # teal,      RGB = 69,160,162
r1_hex = '#e87a59'    # red,       RGB = 232,122,89
tl1_hex = '#7dcaa9'   # teal,      RGB = 125,202,169
g_hex = '#649E7D'     # green,     RGB = 100,158,125
o_hex = '#dc8018'     # orange,    RGB = 220,128,24
tn_hex = '#C89F91'    # tan,       RGB = 200,159,145
g50_hex = '#6c6d6c'   # grey-50,   RGB = 108,109,108
bg_hex = '#4f6268'    # blue grey, RGB = 79,98,104
g25_hex = '#c7cccf'   # grey-25,   RGB = 199,204,207